# Circuit Breakers with Seldon and Ambassador

This notebook shows how you can deploy Seldon Deployments which can have circuit breakers via Ambassador's circuit breakers configuration.


## Setup Seldon Core

Use the setup notebook to [Setup Cluster](../../seldon_core_setup.ipynb#Setup-Cluster) with [Ambassador Ingress](../../seldon_core_setup.ipynb#Ambassador) and [Install Seldon Core](../../seldon_core_setup.ipynb#Install-Seldon-Core). Instructions [also online](./seldon_core_setup.html).

In [1]:
!kubectl create namespace seldon

Error from server (AlreadyExists): namespaces "seldon" already exists


In [2]:
!kubectl config set-context $(kubectl config current-context) --namespace=seldon

Context "docker-desktop" modified.


## Launch main model

We will create a very simple Seldon Deployment with a dummy model image `seldonio/mock_classifier:1.0`. This deployment is named `example`. We will add following circuit breakers configurations.

```
    "seldon.io/ambassador-circuit-breakers-max-connections":"200",
    "seldon.io/ambassador-circuit-breakers-max-pending-requests":"100",
    "seldon.io/ambassador-circuit-breakers-max-requests":"200",
    "seldon.io/ambassador-circuit-breakers-max-retries":"3"
```

Where

  * `"seldon.io/ambassador-circuit-breakers-max-connections":"200"` is the maximum number of connections will make to the Seldon Deployment
  * `"seldon.io/ambassador-circuit-breakers-max-pending-requests":"100"` is the maximum number of requests that will be queued while waiting for a connection
  * `"seldon.io/ambassador-circuit-breakers-max-requests":"200"` is the maximum number of parallel outstanding requests to the Seldon Deployment
  * `"seldon.io/ambassador-circuit-breakers-max-retries":"3"` the maximum number of parallel retries allowed to the Seldon Deployment
  

In [38]:
%%writefile Model.py
import time
class Model:
    def predict(self, data, *args, **kwargs):
        print(f"Received {data}")
        print(f"Sleeping for 30 seconds")
        time.sleep(10)
        print("Done")
        return "done"

Overwriting Model.py


In [39]:
!s2i build . seldonio/seldon-core-s2i-python3:0.18 long_running_model:0.1 \
        -e MODEL_NAME=Model -e API_TYPE=REST -e SERVICE_TYPE=MODEL -e PERSISTENCE=0

---> Installing application source...
Build completed successfully


In [33]:
%%writefile model_circuit_breakers_ambassador.json

{
    "apiVersion": "machinelearning.seldon.io/v1alpha2",
    "kind": "SeldonDeployment",
    "metadata": {
        "labels": {
            "app": "seldon"
        },
        "name": "example"
    },
    "spec": {	
        "name": "production-model",
    "annotations": {
        "seldon.io/rest-timeout":"100000",
        "seldon.io/ambassador-circuit-breakers-max-connections":"1",
        "seldon.io/ambassador-circuit-breakers-max-pending-requests":"1",
        "seldon.io/ambassador-circuit-breakers-max-requests":"1",
        "seldon.io/ambassador-circuit-breakers-max-retries":"3"
    },
        "predictors": [
            {
                "componentSpecs": [{
                    "spec": {
                        "containers": [
                            {
                                "image": "long_running_model:0.1",
                                "imagePullPolicy": "IfNotPresent",
                                "name": "classifier"
                            }
                        ],
                        "terminationGracePeriodSeconds": 1
                    }}
                                  ],
                "graph":
                {
                    "children": [],
                    "name": "classifier",
                    "type": "MODEL",
                    "endpoint": {
                        "type": "REST"
                    }},
                "name": "single",
                "replicas": 1
            }
        ]
    }
}

Overwriting model_circuit_breakers_ambassador.json


In [43]:
!kubectl apply -f model_circuit_breakers_ambassador.json

seldondeployment.machinelearning.seldon.io/example created


In [13]:
!kubectl rollout status deploy/$(kubectl get deploy -l seldon-deployment-id=production-model-example -o jsonpath='{.items[0].metadata.name}')

Waiting for deployment "production-model-single-bf563a0" rollout to finish: 0 of 1 updated replicas are available...
^C


### Let's try sending two requests

The first request should be accepted, but the second one should fail:

First request:
```
curl -v -X POST -H 'Content-Type: application/json' \
    -d '{"data": {"ndarray": ["data"]}}' \
    http://localhost:8003/seldon/default/example/api/v1.0/predictions
```

Now in another terminal you can run the second request:

```
curl -v -X POST -H 'Content-Type: application/json' \
    -d '{"data": {"ndarray": [[39, 7, 1, 1, 1, 1, 4, 1, 2174, 0, 40, 9]]}}' \
    http://localhost:8003/seldon/default/example/api/v1.0/predictions
```

In [42]:
!kubectl delete -f model_circuit_breakers_ambassador.json

seldondeployment.machinelearning.seldon.io "example" deleted
